I was curious about Var. 5:

Var. 5	Fmt: N2	Col: 3507-3508	Name: Q4 (E)
	4b. Our firm's corporate culture: (choose best option)
	1=Is exactly where it should be	3=Needs considerable work to get to where it should be
	2=Needs some work but is close to where it should be	4=Needs a substantial overhaul

What are the major differences between what people who voted 1 and those who voted 4 said about their company's culture? Lets compare.

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
import re
import pickle as pk

import seaborn as sns
import numpy as np
import nltk.tokenize as tokenize

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import cross_validation, grid_search
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn import linear_model, decomposition
import nltk

from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords, words

import matplotlib
matplotlib.style.use('ggplot')

In [2]:
def strfy(x):
    return x.fillna(".").map(str)


df = pd.read_csv('corporate_culture_trunc.csv', encoding='latin-1') #reads csv into dataframe
df = df.dropna(subset=['Q1']) #drop row if no description (NaN in Q1)
dfs = df.Q1
dfs = [dfs1.encode('utf-8') for dfs1 in dfs]
df.Q1 = dfs

##LETS MERGE TEXT TOGETHER
df['merge_text'] = df.Q1.map(str)+strfy(df.Q6_Other)+strfy(df.Q13_5_Other)+strfy(df.Q14_Other)
df['merge_text'] = df['merge_text']+strfy(df.Q14_OE_2)+strfy(df.Q14_OE_3)+strfy(df.Q14_OE_5)+strfy(df.Q14_OE_6)+strfy(df.Q14_OE_7)+strfy(df.Q14_OE_8)+strfy(df.Q14_OE_9)+strfy(df.Q14_OE_11)+strfy(df.Q14_OE_12)

df = df.drop(['Q6_Other','Q13_5_Other','Q14_Other','Q14_OE_1','Q14_OE_2','Q14_OE_3','Q14_OE_4','Q14_OE_5','Q14_OE_6','Q14_OE_7','Q14_OE_8','Q14_OE_9','Q14_OE_10','Q14_OE_9','Q14_OE_10','Q14_OE_11','Q14_OE_12'],axis=1)


In [12]:
df_hl = df.loc[(df["Q4"] == 1) | (df["Q4"] == 4)] #hates or loves the culture
print(len(df_hl))

culture_text=df_hl['Q1'] #takes only the first column
culture_text2=df_hl['merge_text'] #takes text merged from several columns


x_train, x_test, y_train, y_test = cross_validation.train_test_split(culture_text,df_hl['Q4'],test_size=0.2)
x_train2, x_test2, y_train2, y_test2 = cross_validation.train_test_split(culture_text2,df_hl['Q4'],test_size=0.2)

#print(list(df_all.columns.values))

447


In [13]:
mypipeline=Pipeline([
  ('cvect', CountVectorizer(ngram_range=(1, 2), stop_words=nltk.corpus.stopwords.words('english'))),
  ('tfidf', TfidfTransformer()),
  ('rclassify', linear_model.RidgeClassifier())
])

mypipeline.fit(x_train,y_train)
print(mypipeline.score(x_test,y_test))
mypipeline.fit(x_train2,y_train2)
print(mypipeline.score(x_test2,y_test2))

## EIGHTY PERCENT ACCURACY RATE

0.822222222222
0.811111111111


In [14]:
cvect = CountVectorizer(ngram_range=(1, 1), stop_words=nltk.corpus.stopwords.words('english'))
Xvect = cvect.fit_transform(culture_text2)
print(Xvect.shape)[1]
Xvect_names = cvect.get_feature_names()
tfidf = TfidfTransformer()
Xtfidf = tfidf.fit_transform(Xvect)

x_train, x_test, y_train, y_test = cross_validation.train_test_split(Xtfidf,df_hl['Q4'],test_size=0.2)

ridge_reg = linear_model.RidgeClassifier()
ridge_reg.fit(x_train, y_train)
print(ridge_reg.score(x_test,y_test))
xarray = ridge_reg.coef_

xlist=[]
for i in range(xarray.shape[1]):
    if xarray[0][i] != 0:
        xlist.append({'word':Xvect_names[i], 'beta':xarray[0][i], 'absbeta':abs(xarray[0][i])})

2350
0.833333333333


In [128]:
newlist = sorted(xlist, key=lambda k: k['absbeta'],reverse=True)
for news in newlist[0:60]:
    print(news['word'],news['beta'])

(u'open', -0.96424683447305848)
(u'customer', -0.9458448628881686)
(u'integrity', -0.90759968121923895)
(u'lack', 0.86487500171961296)
(u'professional', -0.79923418766377208)
(u'collaborative', -0.78268347244186487)
(u'right', -0.77363298365784494)
(u'toxic', 0.75370931467298963)
(u'change', 0.75226441892105356)
(u'silos', 0.71333925499877859)
(u'healthy', -0.71258612775350672)
(u'opportunity', -0.704686097792809)
(u'fun', -0.69026699532102653)
(u'team', -0.6795651824040676)
(u'unstructured', 0.67756599223692504)
(u'clients', -0.66489897750354654)
(u'respect', -0.65667228078188078)
(u'supportive', -0.65559731556818934)
(u'opaque', 0.65112278124470235)
(u'innovation', -0.64923591429934269)
(u'unstructed', 0.63943630362563741)
(u'autocratic', 0.62639530795070941)
(u'bad', 0.61054144793787324)
(u'political', 0.60908573434982705)
(u'dysfunctional', 0.59905002056498269)
(u'collegial', -0.59655844334909802)
(u'secretive', 0.59639149411615222)
(u'bureaucracy', 0.58345232813185022)
(u'chaos', 

Top words that people who like their company culture used [in order of importance]:

open, customer, integrity, professional, collaborative, right, healthy, opportunity, fun, team, clients, respect, supportive, innovation, collegial, relaxed, play, teamwork, strong, innovative, honesty, excellence, friendly, ethical, collaboration, stable, cfo, empowering, resonant, collegiate, caring, christian, responsibility


Top words that people who didn't like their company culture used [in order of importance]:

lack, toxic, change, silos, unstructured, opaque, unstructed, autocratic, bad, political, dysfunctional, secretive, bureaucracy, chaos, verify, siloed, little, stressful, top, fear, oppressive, low, great, sales, dictatorial, non